# Function Calling

Function calling allows Mistral models to connect to external tools. By integrating Mistral models with external tools such as user defined functions or APIs, users can easily build applications catering to specific use cases and practical problems. In this guide, for instance, we wrote two functions for tracking payment status and payment date. We can use these two tools to provide answers for payment-related queries.

At a glance, there are four steps with function calling:

- User: specify tools and query
- Model: Generate function arguments if applicable
- User: Execute function to obtain tool results
- Model: Generate final answer

In this guide, we will walk through a simple example to demonstrate how function calling works with Mistral models in these four steps.

Before we get started, let’s assume we have a dataframe consisting of payment transactions. When users ask questions about this dataframe, they can use certain tools to answer questions about this data. This is just an example to emulate an external database that the LLM cannot directly access.

In [ ]:
%pip install pandas mistralai

In [1]:
import pandas as pd
import json
import os, dotenv, mistralai

dotenv.load_dotenv()

assert "GITHUB_TOKEN" in os.environ, "Please set the GITHUB_TOKEN environment variable."
github_token = os.environ["GITHUB_TOKEN"]

# We can use some defaults for the other two variables
endpoint = "https://models.inference.ai.azure.com"
model_name = "mistral-nemo"


# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005', 'T1006'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001', 'C003'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20, 120.00],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Unpaid', 'Paid', 'Pending', 'Paid']

}

# Create DataFrame
df = pd.DataFrame(data)

In [3]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Unpaid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending
5,T1006,C003,120.00,2021-10-08,Paid


## Step 1. User: specify tools and query

### Tools

Users can define all the necessary tools for their use cases.

- In many cases, we might have multiple tools at our disposal. For example, let’s consider we have two functions as our two tools: `retrieve_payment_status` and `retrieve_payment_date` to retrieve payment status and payment date given transaction ID.

In [3]:
def retrieve_payment_status(df: pd.DataFrame, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: pd.DataFrame, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})



In [ ]:
import random
def escalate(transaction_id):
    return json.dumps({"ticket_id": str(random.randint(1000000000,9999999999)), "transaction_id": transaction_id})
escalate(10)

'{"ticket_id": "7782500727", "transaction_id": 10}'

In [77]:
from datetime import timedelta, date
end_date = date.today() + timedelta(days=2)
def ticket_details(ticket_id, transaction_id):
    return json.dumps({"ticket_id": ticket_id, "transaction_id": transaction_id, "status": "IN PROGRESS", "eta": end_date.strftime("%Y-%m-%d")})

ticket_details('12313123','T10002')

'{"ticket_id": "12313123", "transaction_id": "T10002", "status": "IN PROGRESS", "eta": "2024-09-07"}'

- In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Since we have two functions here, let’s list two function specifications in a list.

In [78]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Retrieves the payment status of the given transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Retrieves the payment status of the given transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "escalate",
            "description": "Escalate matter to next level and assigns a ticket id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ticket_details",
            "description": "Retrieve the ticket details",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticket_id": {
                        "type": "string",
                        "description": "The ticket id.",
                    },
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id of the ticket"
                    }
                },
                "required": ["transaction_id", "ticket_id"],
            },
        }
    }
]


- Then we organize the two functions into a dictionary where keys represent the function name, and values are the function with the df defined. This allows us to call each function based on its function name.

In [79]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df),
  'escalate': functools.partial(escalate),
  'ticket_details': functools.partial(ticket_details)
}

### User query

Suppose a user asks the following question: “What’s the status of my transaction?” A standalone LLM would not be able to answer this question, as it needs to query the business logic backend to access the necessary data. But what if we have an exact tool we can use to answer this question? We could potentially provide an answer!

In [5]:
from mistralai.models.chat_completion import ChatMessage

messages = [
    ChatMessage(role="user", content="What is the status of my transaction buddy?")
]


## Step 2. Model: Generate function arguments 

How do Mistral models know about these functions and know which function to use? We provide both the user query and the tools specifications to Mistral models. The goal in this step is not for the Mistral model to run the function directly. It’s to 1) determine the appropriate function to use , 2) identify if there is any essential information missing for a function, and 3) generate necessary arguments for the chosen function.

In [6]:
from mistralai.client import MistralClient

model = "mistral-nemo"

client = MistralClient(api_key=github_token, endpoint=endpoint)

response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response.model_dump()

{'id': 'd7586374a919405492fd5facac85b17e',
 'object': 'chat.completion',
 'created': 1725538532,
 'model': 'mistral-nemo',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Sure, please provide me with the transaction id.',
    'name': None,
    'tool_calls': None,
    'tool_call_id': None},
   'finish_reason': <FinishReason.stop: 'stop'>}],
 'usage': {'prompt_tokens': 159, 'total_tokens': 169, 'completion_tokens': 10}}

In [85]:
messages.append(response.choices[0].message)

In [86]:
messages

[ChatMessage(role='user', content='What is the status of my transaction buddy?', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='Sure, please provide me with the transaction id.', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='T1005', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='6zFrRIsme', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))], tool_call_id=None),
 ChatMessage(role='tool', content='{"status": "Pending"}', name='retrieve_payment_status', tool_calls=None, tool_call_id='6zFrRIsme'),
 ChatMessage(role='assistant', content='The status of your transaction with ID T1005 is currently "Pending".', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='When!!???? Are you sure?', name=None, tool_calls=None, tool_call_

In [9]:
messages.append(ChatMessage(role="user", content="T1005"))
response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response.model_dump()

{'id': 'ce26ec58b9f84d90ad6e5840898f9888',
 'object': 'chat.completion',
 'created': 1725538545,
 'model': 'mistral-nemo',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '',
    'name': None,
    'tool_calls': [{'id': '6zFrRIsme',
      'type': <ToolType.function: 'function'>,
      'function': {'name': 'retrieve_payment_status',
       'arguments': '{"transaction_id": "T1005"}'}}],
    'tool_call_id': None},
   'finish_reason': <FinishReason.tool_calls: 'tool_calls'>}],
 'usage': {'prompt_tokens': 177, 'total_tokens': 203, 'completion_tokens': 26}}

## Step 3. User: Execute function to obtain tool results

How do we execute the function? Currently, it is the user’s responsibility to execute these functions and the function execution lies on the user side. In the future, we may introduce some helpful functions that can be executed server-side.

Let’s extract some useful function information from model response including function_name and function_params. It’s clear here that our Mistral model has chosen to use the function `retrieve_payment_status` with the parameter `transaction_id` set to T1001.

In [87]:
import json
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  ticket_details 
function_params:  {'ticket_id': '1557462481', 'transaction_id': 'T1005'}


In [88]:
function_result = names_to_functions[function_name](**function_params)
function_result


'{"ticket_id": "1557462481", "transaction_id": "T1005", "status": "IN PROGRESS", "eta": "2024-09-07"}'

In [89]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result, tool_call_id=tool_call.id))

In [90]:
messages

[ChatMessage(role='user', content='What is the status of my transaction buddy?', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='Sure, please provide me with the transaction id.', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='T1005', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='6zFrRIsme', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))], tool_call_id=None),
 ChatMessage(role='tool', content='{"status": "Pending"}', name='retrieve_payment_status', tool_calls=None, tool_call_id='6zFrRIsme'),
 ChatMessage(role='assistant', content='The status of your transaction with ID T1005 is currently "Pending".', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='When!!???? Are you sure?', name=None, tool_calls=None, tool_call_

## Step 4. Model: Generate final answer

We can now provide the output from the tools to Mistral models, and in return, the Mistral model can produce a customised final response for the specific user.

In [91]:
response = client.chat(
    model=model,
    messages=messages
)

response.choices[0].message.content

'The estimated resolution time for your ticket #1557462481 is September 7, 2024.'

In [35]:
response.choices[0].message.content

"The status of transaction T1006 is 'Paid'."

In [84]:
messages.append(ChatMessage(role="user", content="Can you provide the eta for this ticket?"))
response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
response.choices[0].message.content

''

In [33]:
print(response.choices[0].message.content)

There could be several reasons why your transaction is still pending. Here are a few possibilities:

* The recipient's bank has not yet processed the transaction.
* There is a delay in the banking system.
* The transaction is waiting for confirmation from the recipient's bank.
* The transaction is being reviewed due to suspected fraudulent activity.
* There is an issue with the recipient's account or bank.

If the transaction has been pending for more than a few business days, I would recommend contacting your bank for more information.


In [49]:
response.model_dump()

{'id': '7e7158462344455d9252603df8530674',
 'object': 'chat.completion',
 'created': 1725536665,
 'model': 'mistral-large',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '',
    'name': None,
    'tool_calls': [{'id': 'XPcWO1UPu',
      'type': <ToolType.function: 'function'>,
      'function': {'name': 'retrieve_payment_date',
       'arguments': '{"transaction_id": "T1006"}'}}],
    'tool_call_id': None},
   'finish_reason': <FinishReason.tool_calls: 'tool_calls'>}],
 'usage': {'prompt_tokens': 236, 'total_tokens': 266, 'completion_tokens': 30}}